# Image Classification with CNN using Keras

# Task 1: Import Libraries

In [ ]:
import tensorflow as tf
import os
import numpy as np

from matplotlib import pyplot as plt
%matplotlib inline

if not os.path.isdir('models'):
    os.mkdir('models')
    
print('TensorFlow version:', tf.__version__)
print('Is using GPU?', tf.test.is_gpu_available())

# Task 2: Preprocess Data

In [ ]:
def get_three_classes(x, y):
    indices_0, _ = np.where(y == 0.)
    indices_1, _ = np.where(y == 1.)
    indices_2, _ = np.where(y == 2.)
    indices_3, _ = np.where(y == 3.)
    indices_4, _ = np.where(y == 4.)
    indices_5, _ = np.where(y == 5.)
    indices_6, _ = np.where(y == 6.)
    indices_7, _ = np.where(y == 7.)
    indices_8, _ = np.where(y == 8.)
    indices_9, _ = np.where(y == 9.)
    print(indices_0.shape,indices_1.shape,indices_2.shape)

    indices = np.concatenate([indices_0, indices_1, indices_2, indices_3, indices_4,indices_5,indices_6,indices_7,indices_9,indices_8], axis=0)
    print(indices.shape)
    
    x = x[indices]
    y = y[indices]
    print(x,y)
    
    count = x.shape[0]
    print(count)
    indices = np.random.choice(range(count), count, replace=False)
    print(indices)
    
    x = x[indices]
    y = y[indices]
    print(x,y)
    
    y = tf.keras.utils.to_categorical(y)
    print(y)
    
    return x, y

In [ ]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
x_train, y_train = get_three_classes(x_train, y_train)
x_test, y_test = get_three_classes(x_test, y_test)

print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

# Task 3: Visualize Examples

In [ ]:
class_names = ['airplane','automobile','bird','cat','deer','dog','frog','horse','ship','truck']

def show_random_examples(x ,y ,p):
    indices = np.random.choice(range(x.shape[0]), 12, replace= False)
    
    x = x[indices]
    y = y[indices]
    p = p[indices]
    plt.figure( figsize= (12, 5))
    for i in range(12):
        plt.subplot( 2 , 6 , 1+ i)
        plt.imshow(x[i])
        plt.xticks([])
        plt.yticks([])
        col = 'green' if np.argmax(y[i]) == np.argmax(p[i]) else 'red'
        plt.xlabel(class_names[np.argmax(p[i])], color=col)
    plt.show()

show_random_examples(x_train, y_train, y_train)

In [ ]:
show_random_examples(x_test, y_test, y_test)

# Task 4: Create Model

In [ ]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.layers import Dropout, Flatten, Input, Dense

def create_model():
    def add_conv_block(model, num_filters):
        model.add(Conv2D(num_filters, 3, activation='relu', padding = 'same'))
        model.add(BatchNormalization())
        model.add(Conv2D(num_filters, 3, activation='relu'))
        model.add(MaxPooling2D(pool_size=2))
        model.add(Dropout(0.5))
        return model
    
    model = tf.keras.models.Sequential()
    model.add(Input(shape=(32,32,3)))
    
    model = add_conv_block(model, 32)
    #model = add_conv_block(model, 32)
    model = add_conv_block(model, 64)
    #model = add_conv_block(model, 64)
    model = add_conv_block(model, 128)
    #model = add_conv_block(model, 128)
    
    model.add(Flatten())
    model.add(Dense(10,activation = 'softmax'))
    
    model.compile(
        loss = 'categorical_crossentropy',
        optimizer='adam', metrics=['accuracy']
    )
    
    return model

model = create_model()
model.summary()

# Task 5: Train the Model

In [ ]:
h = model.fit(x_train/255., y_train, validation_data=(x_test/255.,y_test),
             epochs=15, batch_size=128,
             callbacks=[
                 tf.keras.callbacks.EarlyStopping(monitor = 'val_accuracy', patience = 3)
                 
             ]
        )

# Task 6: Final Predictions

In [ ]:
accs = h.history['accuracy']
val_accs = h.history['val_accuracy']

plt.plot(range(len(accs)), accs, label='Training')
plt.plot(range(len(accs)), val_accs, label='Validation')
plt.legend()
plt.show()

In [ ]:
pred = model.predict(x_test/255.)

In [ ]:
show_random_examples(x_test,y_test, pred)